In [1]:
import sys
import os
from dotenv import load_dotenv
import warnings
import nest_asyncio
import pyperclip
nest_asyncio.apply()
# warnings.filterwarnings('ignore')


load_dotenv()


True

In [2]:
from llama_index.llms.anthropic import Anthropic
from llama_index.multi_modal_llms.anthropic import AnthropicMultiModal

In [3]:
llm = Anthropic(model= 'claude-3-5-sonnet-20241022', max_tokens=8192)


In [4]:
coding_agent_prompt = """
Please provide Python code that solves the following problem. The code should:
- Use standard Python syntax and conventions
- Include necessary imports at the beginning
- Be properly indented and formatted
- Include brief inline comments for clarity
- Handle basic error cases
- Return or print the required output

Return only the executable Python code without explanations.
Problem:
{task_description}
"""


error_correction_prompt = """
The following script has an error:

### Original Script:
{original_script}

### Error Message:
{error_string}

Please analyze the error and suggest a corrected version of the script, explaining the necessary changes. Include comments in the code where applicable to explain the corrections.

Provide only the corrected script below:
"""


In [5]:
def remove_non_python(text):
    return text.replace("```python", "").replace("```", "")

In [6]:
def generate_script(task_description, llm=llm, coding_agent_prompt=coding_agent_prompt):
    prompt = coding_agent_prompt.format(task_description=task_description)
    response = llm.complete(prompt)
    return remove_non_python(response.text)

def fix_script(script, result, llm=llm, coding_agent_prompt=coding_agent_prompt):
    prompt = coding_agent_prompt.format(task_description=f"The following script was generated to solve a task, but it did not work. Please correct it: {script}. The result of running the script was: {result}")
    response = llm.complete(prompt)
    return remove_non_python(response.text)

import sys
from io import StringIO

def run_code(code_string):
    try:
        # Capture stdout
        old_stdout = sys.stdout
        redirected_output = sys.stdout = StringIO()
        
        # Create a dictionary to hold local variables
        exec_locals = {}
        
        # Check if the code is an evaluable expression
        try:
            exec(f"_last_expr_result = {code_string}", {}, exec_locals)
            exec_result = exec_locals.get('_last_expr_result')
        except SyntaxError:
            # If it's not an expression, just execute the code
            exec(code_string, {}, exec_locals)
            exec_result = None

        # Get printed output
        printed_output = redirected_output.getvalue()
        
        # Restore stdout
        sys.stdout = old_stdout

        return {
            'is_error': False,
            'result': exec_result,
            'output': printed_output.strip(),
            'error_message': ''
        }
        
    except Exception as e:
        return {
            'is_error': True,
            'result': None,
            'output': '',
            'error_message': str(e)
        }


In [7]:
def run_and_fix(task_description, max_iterations=3, llm=llm, coding_agent_prompt=coding_agent_prompt ):    
    all_run_results = []
    script = generate_script(task_description, llm, coding_agent_prompt)
    run_results = run_code(script)
    run_results['script'] = script
    all_run_results.append(run_results)
    if run_results['is_error']==False:
        run_results['fix_iterations'] = 0
        return all_run_results
    else:
        for i in range(max_iterations):
            script = fix_script(script, run_results['error_message'], llm, coding_agent_prompt)
            run_results = run_code(script)
            run_results['script'] = script
            all_run_results.append(run_results)
            if run_results['is_error']==False:
                run_results['fix_iterations'] = i+1
                all_run_results.append(run_results)
                return all_run_results
            else:
                continue        
    run_results['fix_iterations'] = max_iterations
    all_run_results.append(run_results)
    return all_run_results


def extract_functions(code_string):
    try:
        # Parse code string into AST
        tree = ast.parse(code_string)
        
        functions = {}
        
        # Walk through AST nodes
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                # Get function name
                func_name = node.name
                
                # Get input arguments
                args = []
                for arg in node.args.args:
                    args.append(arg.arg)
                    
                # Find return statements
                returns = []
                for child in ast.walk(node):
                    if isinstance(child, ast.Return):
                        # Extract return value
                        if isinstance(child.value, ast.Name):
                            returns.append(child.value.id)
                        elif isinstance(child.value, ast.Tuple):
                            for elt in child.value.elts:
                                if isinstance(elt, ast.Name):
                                    returns.append(elt.id)
                
                # Store in dictionary
                functions[func_name] = {
                    'arguments': args,
                    'returns': returns
                }
                
        return functions
        
    except SyntaxError:
        return "Invalid Python code"
    except Exception as e:
        return f"Error parsing code: {str(e)}"

    

In [8]:
def run_to_clipboard(task_description):
    all_run_results = run_and_fix(task_description)
    script = all_run_results[-1]['script']
    pyperclip.copy(script)
    return script

In [14]:
a = 'write a function that read a string containing python code and extract all the function definitions as a dictionary with: function_name, input arguments and return arguments'
b = 'write a function that get a function and a jupyter notebook file name and run the function on all the jupyter notebook cells, it return a list with the return values from all the cells'
c = 'write a function that get a jupyter notebook file and save all the functions in the notebook as a python file'
d = 'change this function to save the original notebook file as a bk[i]_original_name.ipynb and move all the functions from the code cell of the notebook to a new python file named functions.py' 
e = 'write a function that get a jupyter notebook file and attach the imports in the start of the file functions.py '

In [15]:
run_to_clipboard(e)

'\nimport json\nimport re\nimport nbformat\n\ndef extract_imports_from_notebook(notebook_path):\n    try:\n        # Read the notebook\n        with open(notebook_path, \'r\', encoding=\'utf-8\') as f:\n            notebook = nbformat.read(f, as_version=4)\n            \n        # Extract imports from code cells\n        imports = []\n        for cell in notebook.cells:\n            if cell.cell_type == \'code\':\n                code = cell.source\n                # Find import statements using regex\n                import_lines = re.findall(r\'^(?:from\\s+[\\w.]+\\s+import\\s+[\\w.*,\\s]+|import\\s+[\\w.,\\s]+)\', \n                                        code, \n                                        re.MULTILINE)\n                imports.extend(import_lines)\n        \n        # Remove duplicates while preserving order\n        unique_imports = []\n        for imp in imports:\n            if imp not in unique_imports:\n                unique_imports.append(imp)\n                \

In [17]:

import json
import re
import nbformat

def extract_imports_from_notebook(notebook_path):
    try:
        # Read the notebook
        with open(notebook_path, 'r', encoding='utf-8') as f:
            notebook = nbformat.read(f, as_version=4)
            
        # Extract imports from code cells
        imports = []
        for cell in notebook.cells:
            if cell.cell_type == 'code':
                code = cell.source
                # Find import statements using regex
                import_lines = re.findall(r'^(?:from\s+[\w.]+\s+import\s+[\w.*,\s]+|import\s+[\w.,\s]+)', 
                                        code, 
                                        re.MULTILINE)
                imports.extend(import_lines)
        
        # Remove duplicates while preserving order
        unique_imports = []
        for imp in imports:
            if imp not in unique_imports:
                unique_imports.append(imp)
                
        return unique_imports
        
    except Exception as e:
        print(f"Error processing notebook: {str(e)}")
        return []

def append_imports_to_functions(notebook_path, functions_path):
    try:
        # Get imports from notebook
        imports = extract_imports_from_notebook(notebook_path)
        
        if not imports:
            print("No imports found in notebook")
            return
            
        # Read existing functions.py content
        try:
            with open(functions_path, 'r') as f:
                existing_content = f.read()
        except FileNotFoundError:
            existing_content = ""
            
        # Combine imports with existing content
        import_text = "\n".join(imports)
        new_content = f"{import_text}\n\n{existing_content}"
        
        # Write back to functions.py
        with open(functions_path, 'w') as f:
            f.write(new_content)
            
        print(f"Successfully added {len(imports)} imports to {functions_path}")
        
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    # Example usage
    notebook_path = "main.ipynb"
    functions_path = "functions.py"
    append_imports_to_functions(notebook_path, functions_path)


Successfully added 5 imports to functions.py


In [12]:

import nbformat
import os
import re

def extract_functions(notebook_path):
    try:
        # Read the notebook
        with open(notebook_path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
            
        # Get original filename without extension
        base_name = os.path.splitext(notebook_path)[0]
        
        # Create backup filename
        i = 1
        while os.path.exists(f'bk{i}_{os.path.basename(notebook_path)}'):
            i += 1
        backup_path = f'bk{i}_{os.path.basename(notebook_path)}'
        
        # Save backup copy
        with open(backup_path, 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
            
        # Extract functions from code cells
        functions = []
        for cell in nb.cells:
            if cell.cell_type == 'code':
                # Find function definitions using regex
                matches = re.finditer(r'def\s+\w+\s*\([^)]*\)\s*:', cell.source)
                for match in matches:
                    # Get the full function definition
                    start = match.start()
                    lines = cell.source[start:].split('\n')
                    
                    # Get indented function body
                    func_lines = [lines[0]]
                    for line in lines[1:]:
                        if line.strip() == '' or line.startswith(' '):
                            func_lines.append(line)
                        else:
                            break
                    functions.append('\n'.join(func_lines))
                    
                    # Remove function from cell
                    cell.source = cell.source.replace('\n'.join(func_lines), '')
                    
        # Write functions to functions.py
        with open('functions.py', 'w', encoding='utf-8') as f:
            f.write('\n\n'.join(functions))
            
        # Save modified notebook
        with open(notebook_path, 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
            
        return True
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return False


In [13]:

import json
import shutil
import os
from pathlib import Path

def backup_and_modify_notebook(notebook_path, functions_to_move):
    """
    Backs up notebook and moves specified functions to new file
    """
    try:
        # Get original filename and create backup name
        notebook_path = Path(notebook_path)
        base_name = notebook_path.stem
        ext = notebook_path.suffix
        
        # Find next available backup number
        i = 1
        while True:
            backup_name = f"bk{i}_{base_name}{ext}"
            if not os.path.exists(backup_name):
                break
            i += 1
            
        # Create backup copy
        shutil.copy2(notebook_path, backup_name)
        
        # Load original notebook
        with open(notebook_path, 'r', encoding='utf-8') as f:
            nb = json.load(f)
            
        # Find cells with specified functions and move them
        cells_to_remove = []
        moved_functions = []
        
        for idx, cell in enumerate(nb['cells']):
            if cell['cell_type'] == 'code':
                code = cell['source']
                if isinstance(code, list):
                    code = ''.join(code)
                    
                for func in functions_to_move:
                    if f"def {func}" in code:
                        moved_functions.append(cell)
                        cells_to_remove.append(idx)
                        break
        
        # Remove moved cells from original notebook
        for idx in sorted(cells_to_remove, reverse=True):
            nb['cells'].pop(idx)
            
        # Save modified original notebook
        with open(notebook_path, 'w', encoding='utf-8') as f:
            json.dump(nb, f, indent=2)
            
        # Load backup notebook and append moved functions
        with open(backup_name, 'r', encoding='utf-8') as f:
            backup_nb = json.load(f)
            
        backup_nb['cells'].extend(moved_functions)
        
        # Save modified backup notebook
        with open(backup_name, 'w', encoding='utf-8') as f:
            json.dump(backup_nb, f, indent=2)
            
        return True
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return False
